# Add trial labels

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/nancy/projects/reward_competition_extention'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

## Inputs & Data

Explanation of each input and where it comes from.

In [7]:
subject_start_stop_frames = pd.read_excel("rce_per_subject_start_stop_video_frame.xlsx")
subject_start_stop_frames = subject_start_stop_frames.dropna(subset=["file_path"])

In [8]:
subject_start_stop_frames["sleap_name"] = subject_start_stop_frames["file_path"].apply(lambda x: os.path.basename(x))
subject_start_stop_frames["video_name"] = subject_start_stop_frames["file_path"].apply(lambda x: ".".join(os.path.basename(x).split(".")[:2]))
subject_start_stop_frames["start_frame"] = subject_start_stop_frames["start_frame"].astype(int)
subject_start_stop_frames["stop_frame"] = subject_start_stop_frames["stop_frame"].astype(int)

In [9]:
subject_start_stop_frames = subject_start_stop_frames.drop(columns=["file_path", "notes", "in_video_subjects"], errors="ignore")

In [10]:
subject_start_stop_frames["video_name"].unique()

array(['20221214_125409_om_and_comp_6_1_and_6_3.1',
       '20221215_145401_comp_amd_om_6_1_and_6_3.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.3',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.3',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.4',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.1',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.2',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.1',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.2',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '

In [11]:
subject_start_stop_frames.head()

,start_frame,stop_frame,tracked_subject,sleap_name,video_name
1,1,25000,6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1
2,27500,73600,6.1_6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1
3,51500,76454,6.3,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1
4,1,48500,6.1_6.3,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1
5,41000,79050,1.1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...


In [12]:
subject_start_stop_frames["current_subject"] = subject_start_stop_frames["tracked_subject"].apply(lambda x: str(x).split("_"))

In [13]:
subject_start_stop_frames = subject_start_stop_frames.explode("current_subject")

In [14]:
subject_start_stop_frames.head()

,start_frame,stop_frame,tracked_subject,sleap_name,video_name,current_subject
1,1,25000,6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1,6.3
2,27500,73600,6.1_6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1,6.1
2,27500,73600,6.1_6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1,6.3
3,51500,76454,6.3,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1,6.3
4,1,48500,6.1_6.3,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1,6.1


In [15]:
TRIAL_LABELS_DF = pd.read_excel("./rce_pilot_2_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = TRIAL_LABELS_DF.rename(columns={col: col.strip().lower() for col in TRIAL_LABELS_DF.columns})
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition"])
TRIAL_LABELS_DF["tone_start_timestamp"] = TRIAL_LABELS_DF["tone_start_timestamp"].astype(np.int64)
TRIAL_LABELS_DF["tone_stop_timestamp"] = TRIAL_LABELS_DF["tone_stop_timestamp"].astype(np.int64)
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: ".".join(os.path.basename(x).split(".")[:2]))

TRIAL_LABELS_DF["tone_start_frame"] = TRIAL_LABELS_DF["tone_start_frame"].astype(int)
TRIAL_LABELS_DF["reward_start_frame"] = TRIAL_LABELS_DF["reward_start_frame"].astype(int)
TRIAL_LABELS_DF["tone_stop_frame"] = TRIAL_LABELS_DF["tone_stop_frame"].astype(int)


TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in TRIAL_LABELS_DF.columns if "unnamed" in col], errors="ignore")

In [16]:
TRIAL_LABELS_DF.head()

,video_name,tone_start_frame,reward_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,condition,competition_closeness,notes,experiment,session_dir,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
1,20230612_101430_standard_comp_to_training_D1_s...,980,1060,1181,[1028 1031] [1149 1266],NaN,1.4,Subj 2 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,982229,1182226,[1030229 1033226] [1151634 1269428],NaN
2,20230612_101430_standard_comp_to_training_D1_s...,3376,3456,3575,[3545 3545] [3547 3549] [3550 3554] [3554 3556...,NaN,1.3,Subj 1 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,3382227,3582224,[3550827 3551624] [3553824 3555829] [3556426 3...,NaN
3,20230612_101430_standard_comp_to_training_D1_s...,5672,5752,5871,[5761 5762] [5762 5942],NaN,1.4,Subj 2 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,5682225,5882222,[5771223 5772822] [5773422 5952622],NaN
4,20230612_101430_standard_comp_to_training_D1_s...,7468,7548,7668,[7632 7634] [7635 7665] [7665 7945],NaN,1.4,Subj 2 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,7482224,7682221,[7647221 7648224] [7649024 7679421] [7680023 7...,NaN
5,20230612_101430_standard_comp_to_training_D1_s...,8566,8646,8765,[8678 8733] [8735 8975],NaN,1.4,Subj 2 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,8582220,8782223,[8694220 8750020] [8750420 8992222],NaN


In [17]:
merged_df = pd.merge(subject_start_stop_frames, TRIAL_LABELS_DF, on="video_name", how="inner")

In [18]:
merged_df = merged_df.dropna(subset=["current_subject"])

merged_df = merged_df[(merged_df['tone_start_frame'] > merged_df['start_frame']) & (merged_df['tone_start_frame'] < merged_df['stop_frame'])]

In [19]:
merged_df = merged_df.drop(columns=["start_frame", "stop_frame"], errors="ignore")
merged_df = merged_df.drop(columns=["first_timestamp", "last_timestamp"], errors="ignore")
merged_df = merged_df.drop(columns=["box_1_port_entry_frames", "box_2_port_entry_frames"], errors="ignore")
merged_df = merged_df.drop(columns=['box_1_port_entry_timestamps', 'box_2_port_entry_timestamps'], errors="ignore")


In [20]:
merged_df = merged_df.sort_values(by=["session_dir", "current_subject", "tone_start_timestamp"]).reset_index(drop=True)

In [21]:
merged_df.columns

Index(['tracked_subject', 'sleap_name', 'video_name', 'current_subject',
       'tone_start_frame', 'reward_start_frame', 'tone_stop_frame',
       'condition', 'competition_closeness', 'notes', 'experiment',
       'session_dir', 'all_subjects', 'tone_start_timestamp',
       'tone_stop_timestamp'],
      dtype='object')

In [22]:
merged_df.head()

,tracked_subject,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,session_dir,all_subjects,tone_start_timestamp,tone_stop_timestamp
0,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1205,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741
1,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3599,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740
2,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5895,6014,1.2,Subj 2 blocking Subj 1,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",5826740,6026737
3,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7692,7811,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",7626736,7826735
4,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8789,8910,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",8726734,8926734


In [23]:
non_competitive_labels = ['rewarded', 'omission', 'both_rewarded']

In [24]:
merged_df["current_subject"] = merged_df["current_subject"].astype(str)
merged_df["condition"] = merged_df["condition"].astype(str)

In [25]:
merged_df["trial_label"] = merged_df.apply(lambda x: x["condition"] if x["condition"] in non_competitive_labels else ("win" if x["condition"] == x["current_subject"] else "lose"), axis=1)

In [26]:
merged_df

,tracked_subject,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,session_dir,all_subjects,tone_start_timestamp,tone_stop_timestamp,trial_label
0,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1205,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741,win
1,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3599,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740,lose
2,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5895,6014,1.2,Subj 2 blocking Subj 1,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",5826740,6026737,lose
3,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7692,7811,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",7626736,7826735,lose
4,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8789,8910,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",8726734,8926734,lose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,1.1_1.4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,1.4,65186,65266,65386,both_rewarded,NaN,NaN,both_rewarded,20230625_112913_standard_comp_to_both_rewarded...,"['1.1', '1.4']",64304011,64504015,both_rewarded
825,1.1_1.4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,1.4,66684,66764,66883,both_rewarded,NaN,NaN,both_rewarded,20230625_112913_standard_comp_to_both_rewarded...,"['1.1', '1.4']",65804029,66004033,both_rewarded
826,1.1_1.4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,1.4,68680,68760,68879,both_rewarded,NaN,NaN,both_rewarded,20230625_112913_standard_comp_to_both_rewarded...,"['1.1', '1.4']",67804057,68004058,both_rewarded
827,1.1_1.4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,1.4,70576,70656,70775,both_rewarded,NaN,NaN,both_rewarded,20230625_112913_standard_comp_to_both_rewarded...,"['1.1', '1.4']",69704075,69904080,both_rewarded


In [27]:
merged_df.to_excel("rce2_finalized_trial_labels.xlsx", index=False)

# TODO
- Merge recording file name
- Make a dictionary of trial labels 

In [28]:
raise ValueError()

ValueError: 

In [ ]:
import ast

In [ ]:
ast.literal_eval(merged_df["box_1_port_entry_timestamps"].unique()[0])

([1020481, 1061883],
 [1073480, 1074878],
 [1076081, 1167078],
 [1173678, 1252878])

In [ ]:
merged_df["all_subjects"].apply(lambda x: str(x)).unique()

array([], dtype=object)

In [ ]:
merged_df["condition"].apply(lambda x: str(x)).unique()

array([], dtype=object)

In [ ]:
raise ValueError()

ValueError: 

In [ ]:
TRIAL_LABELS_DF["video_name"].iloc[0]

In [ ]:
TRIAL_LABELS_DF["video_file"].unique()

In [ ]:
trodes_metadata_df = pd.read_pickle("./rce_pilot_2_00_trodes_metadata.pkl")
trodes_metadata_df["video_name"] = trodes_metadata_df["video_name"].apply(lambda x: ".".join(os.path.basename(x).split(".")[:2]))

In [ ]:
trodes_metadata_df.head()

In [ ]:
trodes_metadata_df["video_name"].iloc[0]

In [ ]:
merged_df = pd.merge(trodes_metadata_df, subject_start_stop_frames, left_on=["video_name", "current_subject"], right_on=["video_name", "current_subject"], how="inner")

In [ ]:
merged_df

In [ ]:
raise ValueError()

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [ ]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [ ]:
FULL_LFP_TRACES_PKL = "{}_03_spectral_trial_labels.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.head()

In [ ]:
TRIAL_LABELS_DF.shape

In [ ]:
# grouped = TRIAL_LABELS_DF.groupby(['recording_file', 'video_file'])['time_stamp_index'].apply(lambda x: sorted(list(x))).reset_index()


grouped = TRIAL_LABELS_DF.groupby(['session_dir', 'video_file']).agg({'time_stamp_index': lambda x: list(x), 'condition': lambda x: list(x), 'competition_closeness': lambda x: list(x), 'video_frame': lambda x: list(x)}).reset_index()


In [ ]:
grouped.head()

In [ ]:
grouped.shape

In [ ]:
grouped["time_stamp_index"].iloc[0]

In [ ]:
grouped["video_file"] = grouped["video_file"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))
VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [ ]:
grouped["video_file"].unique()

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"].unique()

In [ ]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.merge(VIDEO_TO_FRAME_AND_SUBJECT_DF, grouped, how='inner', left_on=['session_dir', 'video_name'], right_on=['session_dir', 'video_file'])

In [ ]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF.rename(columns={'time_stamp_index': 'trial_time_stamps', 'condition': 'trial_label', 'competition_closeness': 'trial_competition_closeness', "video_frame": "trial_frame"}, errors="ignore")

In [ ]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF.head()

In [ ]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_time_stamps"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_time_stamps"].apply(lambda x: np.array(x))
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].apply(lambda x: np.array(x))
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].apply(lambda x: np.array(x))
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_frame"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_frame"].apply(lambda x: np.array(x))

In [ ]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].iloc[0]

In [ ]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].iloc[0]

- Converting the trial label to win or lose based on who won the trial

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.columns

In [ ]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].apply(
    lambda x: ["win" if str(label).strip() == str(x["subject"]).strip() else "lose" if str(label).strip() in x["all_subjects"] else label for label in x["trial_label"]], 
    axis=1)

- Adding the competition closeness as a column

In [ ]:
all_competition_closeness = np.unique(np.concatenate(VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"]))

In [ ]:
raise ValueError()

In [ ]:
# competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].iloc[0]}
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else ("comp" if "subj" in k.lower() else np.nan) for k in all_competition_closeness}

In [ ]:
competition_closeness_map

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].apply(lambda x: [competition_closeness_map[label] for label in x])

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.to_pickle("./proc/rce_pilot_2_05_lfp_spectral_sleap_spikes_labels.pkl")

In [ ]:
raise ValueError("This is not done yet")

In [ ]:
ALL_PHY_DIR

In [ ]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

In [ ]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

- Combining all the unit info dataframes and adding the recording name

In [ ]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [ ]:
recording_to_cluster_info_df.head()

- Filtering for the good units

In [ ]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [ ]:
good_unit_cluster_info_df.head()

In [ ]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [ ]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [ ]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

In [ ]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

- The times that all the spikes happened

In [ ]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [ ]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

In [ ]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

### Combining everything into a dataframe

In [ ]:
recording_to_spike_df = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

In [ ]:
cluster_info_df.head()

In [ ]:
spike_times

In [ ]:
spike_clusters

- Combining the spike time df for all recordings

In [ ]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [ ]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [ ]:
all_spike_time_df.head()

In [ ]:
all_spike_time_df.tail()

## Grouping all the neurons by recording

In [ ]:
# Grouping all spike times by neuron and recording

grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: sorted(list(x))).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: list(x)}).reset_index()

In [ ]:
grouped_df

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.merge(VIDEO_TO_FRAME_AND_SUBJECT_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

## Calculating the firing rates

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.columns

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.head()

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times), stop_time=x["start_stop_timestamps"][1], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_timestamps"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0], stop_time=x["start_stop_timestamps"][1], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5,:10]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5,10:20]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5,20:30]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5, 30:]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_timestamps"].iloc[0].shape

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"].iloc[0].shape

In [ ]:
np.max(VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"].iloc[0])

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError("")